In [4]:
# รันใน Jupyter Cell
!uv pip install torch transformers datasets accelerate peft trl bitsandbytes
!uv pip install wandb huggingface_hub

Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Audited 7 packages in 9ms
Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Resolved 31 packages in 155ms                                        
Prepared 5 packages in 356ms                                             
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 74ms                                
 + gitdb==4.0.12
 + gitpython==3.1.46
 + sentry-sdk==2.51.0
 + smmap==5.0.2
 + wandb==0.24.1


In [3]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install -U trl transformers peft

In [ ]:



try:
    if MY_WANDB_KEY:
        import wandb
        wandb.login(key=MY_WANDB_KEY)
        print("W&B logged in successfully!")
    else:
        print("W&B Key is empty. Skipping W&B login.")
except Exception as e:
    print(f"Failed to login to W&B: {e}")

# --- 3. Login Hugging Face ---
try:
    if MY_HF_TOKEN:
        from huggingface_hub import login
        login(token=MY_HF_TOKEN)
        print("Hugging Face logged in successfully!")
    else:
        print("HF Token is empty. Skipping HF login.")
except Exception as e:
    print(f"Failed to login to Hugging Face: {e}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc
wandb: Currently logged in as: thabunsri42 (thabunsri42-suranaree-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B logged in successfully!
Hugging Face logged in successfully!


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments
)
from peft import LoraConfig
from transformers.trainer_utils import get_last_checkpoint
from trl import SFTTrainer, SFTConfig

# --- Configuration ---
MODEL_ID = "google/gemma-3n-E4B-it"
DATASET_PATH = "gemma_ready_dataset.json"
NEW_MODEL_NAME = "gemma-3n-sales-agent-v1"
YOUR_HF_USERNAME = "Phonsiri"
HUB_REPO_ID = f"{YOUR_HF_USERNAME}/{NEW_MODEL_NAME}"
OUTPUT_DIR = "./results"

# --- 1. Load Tokenizer & Model (H100 + SDPA) ---
print("Loading Tokenizer & Model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa",   # <--- เปลี่ยนกลับมาใช้ SDPA (เร็วและชัวร์บน H100)
    device_map="auto",
)

# --- 2. Prepare Dataset ---
print("Preparing Dataset...")
dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

def format_chat_template(row):
    formatted_text = ""
    for message in row["messages"]:
        role = message["role"]
        content = message["content"]
        if role == "user":
            formatted_text += f"<start_of_turn>user\n{content}<end_of_turn>\n"
        elif role == "model" or role == "assistant": 
            formatted_text += f"<start_of_turn>model\n{content}<end_of_turn>\n"
        else:
            formatted_text += f"<start_of_turn>{role}\n{content}<end_of_turn>\n"
    formatted_text += tokenizer.eos_token
    row["text"] = formatted_text
    return row

dataset = dataset.map(format_chat_template)
print("Removing raw 'messages' column...")
dataset = dataset.remove_columns("messages") 

def formatting_prompts_func(batch):
    return batch["text"]

# --- 3. LoRA Configuration ---
peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

# --- 4. Training Arguments (H100 Config) ---
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=4, # ถ้า H100 แรมเหลือๆ ลองปรับเป็น 32 ได้ครับ
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    bf16=True,
    tf32=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    optim="adamw_torch_fused",
    report_to="wandb",
    run_name=f"{NEW_MODEL_NAME}-H100-run",
    max_grad_norm=1.0,
    warmup_steps=100,
    group_by_length=True,
    push_to_hub=True,
    hub_model_id=HUB_REPO_ID,
    hub_strategy="every_save",
    max_length=4096,
    packing=False,
    dataset_text_field="text"
)

# --- 5. Trainer ---
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    processing_class=tokenizer
)

# --- 6. Resume & Train ---
print("🚀 Starting training on H100 (SDPA Mode)...")
last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
if last_checkpoint:
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

# --- 7. Save ---
print("Pushing final model...")
trainer.save_model(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)
trainer.push_to_hub()
tokenizer.push_to_hub(HUB_REPO_ID)

Loading Tokenizer & Model...


Loading weights:   0%|          | 0/1676 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Removing raw 'messages' column...


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


...

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Phonsiri/gemma-3n-sales-agent-v1/commit/87fb2254bc69c444993c96ae6db08dd10ebb3511', commit_message='Upload tokenizer', commit_description='', oid='87fb2254bc69c444993c96ae6db08dd10ebb3511', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Phonsiri/gemma-3n-sales-agent-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='Phonsiri/gemma-3n-sales-agent-v1'), pr_revision=None, pr_num=None)

In [11]:
import json

# ==========================================
# 1. ฟังก์ชันจำลองระบบหลังบ้าน (รองรับหลาย Tools)
# ==========================================
def mock_tool_execution(tool_name, args):
    print(f"\n[System] ⚙️ กำลังรันเครื่องมือ: {tool_name}")
    print(f"[System] 📝 ข้อมูลนำเข้า: {args}")
    
    # --- Case 1: เช็คสต็อก ---
    if tool_name == "check_stock":
        if "น้ำมันเครื่อง" in args.get("part_name", ""):
            return {
                "status": "available",
                "stock": 50,
                "brands": [
                    {"name": "Shell Helix Ultra", "price": 1250},
                    {"name": "PTT Performa", "price": 950}
                ]
            }
        else:
            # ข้อมูลสมมติทั่วไป
            return {
                "status": "available",
                "stock": 5,
                "brands": [{"name": "Genuine Parts", "price": 1500}]
            }

    # --- Case 2: สร้างออเดอร์ (สั่งซื้อ) ---
    elif tool_name == "create_order":
        # จำลองการสร้างเลข Order
        return {
            "order_id": "ORD-2024-8899",
            "status": "success",
            "message": "สร้างรายการสั่งซื้อเรียบร้อยแล้ว",
            "total_price": 2450,
            "shipping_status": "pending"
        }

    # --- Case 3: เช็คว่าใส่ด้วยกันได้ไหม ---
    elif tool_name == "check_compatibility":
        # สมมติว่าถ้าปีเก่ากว่า 2010 จะใส่ไม่ได้
        year = args.get("year", 2020)
        if year < 2010:
            return {
                "compatible": False,
                "reason": "รุ่นปีเก่าเกินไป ต้องใช้อะไหล่รหัส B-002 แทนครับ"
            }
        else:
            return {
                "compatible": True,
                "note": "สามารถใส่ได้พอดีตรงรุ่นครับ"
            }
            
    # --- Case 4: หาของ ---
    elif tool_name == "search_parts":
        return {
            "results": ["โช้คอัพหน้า Kayaba", "โช้คอัพหลัง Tokico", "ยางกันฝุ่นโช้ค"]
        }

    else:
        return {"error": "Unknown tool"}

# ==========================================
# 2. ฟังก์ชันรัน Agent (Logic เดิม แต่เปลี่ยนไปเรียก mock ใหม่)
# ==========================================
def run_test_scenario(customer_input):
    print("=" * 60)
    print(f"👤 ลูกค้า: {customer_input}")
    
    # 1. Prepare
    messages = [{"role": "user", "content": f"{SYSTEM_PROMPT}\n\nลูกค้า: {customer_input}"}]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True, return_dict=True).to(model.device)
    
    # 2. First Generation
    outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.2, top_p=0.9)
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    print(f"🤖 Agent (คิด): {response}")
    
    # 3. Tool Execution
    if "<tool_call>" in response:
        try:
            json_str = response.split("<tool_call>")[1].split("</tool_call>")[0]
            tool_data = json.loads(json_str)
            
            # เรียกใช้ฟังก์ชัน Mock ใหม่ที่เราเขียน
            tool_result = mock_tool_execution(tool_data["name"], tool_data["arguments"])
            
            # 4. Second Generation (สรุปผล)
            messages.append({"role": "model", "content": response})
            messages.append({"role": "user", "content": f'<tool_response name="{tool_data["name"]}">{json.dumps(tool_result, ensure_ascii=False)}</tool_response>'})
            
            inputs_v2 = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True, return_dict=True).to(model.device)
            outputs_v2 = model.generate(**inputs_v2, max_new_tokens=512, do_sample=True, temperature=0.7)
            final_ans = tokenizer.decode(outputs_v2[0][inputs_v2['input_ids'].shape[1]:], skip_special_tokens=True)
            print(f"🤖 Agent (ตอบ): {final_ans}")
            
        except Exception as e:
            print(f"❌ Error: {e}")

In [12]:
# --- Scenario 1: การสั่งซื้อสินค้า (ต้องเรียก create_order) ---
# โจทย์: ลูกค้าตกลงซื้อ ให้ชื่อและเบอร์โทรมา
run_test_scenario("โอเคครับ เอาผ้าเบรกหน้า Honda Civic 1 ชุด ส่งที่คุณสมชาย ใจดี โทร 081-234-5678")

# --- Scenario 2: การเช็คความเข้ากันได้ (ต้องเรียก check_compatibility) ---
# โจทย์: ลูกค้าไม่แน่ใจว่าของจะใส่กับรถปีเก่าได้ไหม
run_test_scenario("ไฟหน้ารุ่นนี้ใส่กับ Vios ปี 2008 ได้ไหมครับ?")

# --- Scenario 3: ถามหาน้ำมันเครื่อง (ต้องเรียก check_stock) ---
# โจทย์: ถามราคาสินค้าหมวดอื่น
run_test_scenario("มีน้ำมันเครื่องสังเคราะห์แท้แนะนำไหมครับ ราคาเท่าไหร่")

👤 ลูกค้า: โอเคครับ เอาผ้าเบรกหน้า Honda Civic 1 ชุด ส่งที่คุณสมชาย ใจดี โทร 081-234-5678
🤖 Agent (คิด): <tool_call>{"name": "create_order", "arguments": {"items": [{"part_name": "ผ้าเบรกหน้า", "quantity": 1}], "customer_name": "สมชาย ใจดี", "phone": "081-234-5678"}}</tool_call>

[System] ⚙️ กำลังรันเครื่องมือ: create_order
[System] 📝 ข้อมูลนำเข้า: {'items': [{'part_name': 'ผ้าเบรกหน้า', 'quantity': 1}], 'customer_name': 'สมชาย ใจดี', 'phone': '081-234-5678'}
🤖 Agent (ตอบ): สร้างรายการสั่งซื้อเรียบร้อยแล้วครับ ออเดอร์ ID ORD-2024-8899 รวม 2450 บาท กำลังดำเนินการจัดส่งครับ
👤 ลูกค้า: ไฟหน้ารุ่นนี้ใส่กับ Vios ปี 2008 ได้ไหมครับ?
🤖 Agent (คิด): <tool_call>{"name": "check_compatibility", "arguments": {"part_name": "ไฟหน้า", "car_model": "Vios", "year": 2008}}</tool_call>

[System] ⚙️ กำลังรันเครื่องมือ: check_compatibility
[System] 📝 ข้อมูลนำเข้า: {'part_name': 'ไฟหน้า', 'car_model': 'Vios', 'year': 2008}
🤖 Agent (ตอบ): ไฟหน้าแบบนั้นไม่เข้ากับ Vios ปี 2008 ครับ ต้องใช้อะไหล่รหัส B-002 ที่ออก

In [13]:
import json

def mock_tool_execution(tool_name, args):
    print(f"\n[System] ⚙️ กำลังรันเครื่องมือ: {tool_name}")
    print(f"[System] 📝 ข้อมูลนำเข้า: {args}")
    
    # --- Case สั่งหลายอย่าง ---
    if tool_name == "create_order":
        items = args.get("items", [])
        total_qty = sum([i.get("quantity", 1) for i in items])
        item_names = ", ".join([i.get("part_name", "") for i in items])
        
        return {
            "order_id": "ORD-VIP-999",
            "status": "success",
            "summary": f"รับยอดสั่งซื้อ {len(items)} รายการ ({item_names}) รวม {total_qty} ชิ้น",
            "total_price": 5000 * len(items), # สมมติราคา
            "shipping_status": "preparing"
        }

    # --- Case ค้นหา (Search) ---
    elif tool_name == "search_parts":
        category = args.get("category", "")
        keyword = args.get("keyword", "")
        
        if category == "suspension" or "ช่วงล่าง" in keyword:
            return {
                "results": [
                    {"name": "ปีกนกล่าง (Lower Arm)", "price": 3500},
                    {"name": "ลูกหมากกันโคลง", "price": 850},
                    {"name": "ยางกันฝุ่นเพลา", "price": 450}
                ],
                "count": 3
            }
        else:
            return {"results": [], "message": "ไม่พบสินค้าในหมวดที่ระบุ"}

    # --- Case เช็คสต็อก (Check Stock) ---
    elif tool_name == "check_stock":
        part = args.get("part_name", "")
        if "หม้อน้ำ" in part:
            return {"status": "available", "stock": 2, "brands": [{"name": "Koyorad", "price": 4500}]}
        elif "กรองอากาศ" in part:
             return {"status": "available", "stock": 100, "brands": [{"name": "Sakura", "price": 250}]}
        else:
            return {"status": "out_of_stock"}

    # --- Case เช็ค Compatibility ---
    elif tool_name == "check_compatibility":
        return {"compatible": True, "note": "ใส่ได้ครับ ตรงรุ่นเลย"}

    else:
        return {"error": "Unknown tool"}

In [14]:
# --- Scenario 1: สั่งของหลายอย่างในรวดเดียว (JSON Array Test) ---
# โจทย์: ดูว่าโมเดลแยก "ผ้าเบรก 2" กับ "จานเบรก 2" ออกจากกันได้ไหม และดึงเบอร์โทรถูกไหม
print("\n🔹 TEST 1: สั่งของหลายรายการ")
run_test_scenario("รบกวนเปิดออเดอร์ ผ้าเบรกหน้า 2 ชุด แล้วก็จานเบรกหลังอีก 2 คู่ครับ ส่งที่คุณสมศักดิ์ 089-123-4567")

# --- Scenario 2: ค้นหาแบบกว้างๆ (Category Search Test) ---
# โจทย์: ลูกค้าไม่ได้บอกชื่ออะไหล่ แต่บอกว่า "ช่วงล่างดัง" (โมเดลต้องฉลาดเลือก tool 'search_parts')
print("\n🔹 TEST 2: ลูกค้าไม่รู้อะไหล่ (บอกแค่อาการ)")
run_test_scenario("พี่ครับ รถผมช่วงล่างดังกุกกักๆ เวลาตกหลุม มีอะไหล่พวกช่วงล่าง Civic FD แนะนำมั้ย")

# --- Scenario 3: ประโยคซับซ้อน/มีน้ำเยอะ (Noise Filtering Test) ---
# โจทย์: ลูกค้าเล่าสาธยายยาวเหยียด แต่ใจความคือ "เช็คราคาหม้อน้ำ"
print("\n🔹 TEST 3: ประโยคยาว (จับใจความ)")
run_test_scenario("คือเมื่อวานขับรถไปต่างจังหวัดแล้วความร้อนมันขึ้นอะครับ ช่างบอกหม้อน้ำรั่ว ซ่อมไม่ได้แล้ว ต้องเปลี่ยนใหม่เลย อยากรู้ว่ามีหม้อน้ำ Toyota Camry ACV40 ปี 2010 ขายมั้ยครับ ราคาเท่าไหร่")


🔹 TEST 1: สั่งของหลายรายการ
👤 ลูกค้า: รบกวนเปิดออเดอร์ ผ้าเบรกหน้า 2 ชุด แล้วก็จานเบรกหลังอีก 2 คู่ครับ ส่งที่คุณสมศักดิ์ 089-123-4567
🤖 Agent (คิด): <tool_call>{"name": "create_order", "arguments": {"items": [{"part_name": "ผ้าเบรกหน้า", "quantity": 2}, {"part_name": "จานเบรกหลัง", "quantity": 2}], "customer_name": "สมศักดิ์", "phone": "089-123-4567"}}</tool_call>

[System] ⚙️ กำลังรันเครื่องมือ: create_order
[System] 📝 ข้อมูลนำเข้า: {'items': [{'part_name': 'ผ้าเบรกหน้า', 'quantity': 2}, {'part_name': 'จานเบรกหลัง', 'quantity': 2}], 'customer_name': 'สมศักดิ์', 'phone': '089-123-4567'}
🤖 Agent (ตอบ): ออเดอร์ VIP ของคุณสมศักดิ์ ได้รับที่ระบุแล้วครับ ออเดอร์ ID: ORD-VIP-999 รวม 10,000 บาท กำลังเตรียมจัดส่งครับ

🔹 TEST 2: ลูกค้าไม่รู้อะไหล่ (บอกแค่อาการ)
👤 ลูกค้า: พี่ครับ รถผมช่วงล่างดังกุกกักๆ เวลาตกหลุม มีอะไหล่พวกช่วงล่าง Civic FD แนะนำมั้ย
🤖 Agent (คิด): <tool_call>{"name": "search_parts", "arguments": {"keyword": "ช่วงล่าง", "category": "suspension"}}</tool_call>

[System] ⚙️ กำลั

In [15]:
# 1. แก้ System Prompt เพิ่ม Tool ใหม่ (calculate_installment)
NEW_SYSTEM_PROMPT = """คุณเป็น AI ผู้ช่วยขายอะไหล่รถยนต์
คุณมีเครื่องมือ (Tools) ให้ใช้งานดังนี้:
[
  {
    "name": "check_stock",
    "description": "เช็คสต็อกและราคาอะไหล่",
    "parameters": { "type": "object", "properties": { "part_name": { "type": "string" } }, "required": ["part_name"] }
  },
  {
    "name": "calculate_installment",
    "description": "คำนวณค่างวดผ่อนชำระสินค้า",
    "parameters": {
      "type": "object",
      "properties": {
        "total_price": { "type": "integer", "description": "ราคาสินค้ารวม" },
        "months": { "type": "integer", "description": "จำนวนเดือนที่ต้องการผ่อน (3, 6, 10)" }
      },
      "required": ["total_price", "months"]
    }
  }
]
ถ้าต้องใช้เครื่องมือ ให้ตอบกลับด้วย XML tag: <tool_call>{"name": "...", "arguments": {...}}</tool_call>
"""

# 2. จำลอง Mock Tool ใหม่
def mock_new_tool_execution(tool_name, args):
    if tool_name == "calculate_installment":
        price = args.get("total_price", 0)
        months = args.get("months", 10)
        monthly = (price * 1.05) / months # สมมติดอกเบี้ย 5%
        return {
            "monthly_payment": int(monthly),
            "total_payment": int(price * 1.05),
            "interest_rate": "5%"
        }
    # (Tools เก่า)
    elif tool_name == "check_stock":
        return {"status": "available", "price": 20000} 
    return {"error": "Unknown"}

# 3. ลองทดสอบ
print("=" * 60)
print("🧪 ทดสอบ Tool ใหม่ที่ไม่เคยเทรน (calculate_installment)")
print("=" * 60)

customer_input = "ล้อแม็กชุดละ 20,000 บาท ถ้าผ่อน 10 เดือน ตกเดือนละเท่าไหร่ครับ?"
print(f"👤 ลูกค้า: {customer_input}")

# --- Generate ---
messages = [{"role": "user", "content": f"{NEW_SYSTEM_PROMPT}\n\nลูกค้า: {customer_input}"}]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True, return_dict=True).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.1) # temperature ต่ำๆ จะช่วยให้ทำตามคำสั่งแม่นขึ้น
response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
print(f"🤖 Agent (คิด): {response}")

# --- Execution ---
if "<tool_call>" in response:
    try:
        json_str = response.split("<tool_call>")[1].split("</tool_call>")[0]
        tool_data = json.loads(json_str)
        
        # ดูซิว่ามันเรียกชื่อ Tool ถูกไหม?
        if tool_data["name"] == "calculate_installment":
            print("✅ SUCCESS! โมเดลเรียก Tool ใหม่ได้ถูกต้อง")
            result = mock_new_tool_execution(tool_data["name"], tool_data["arguments"])
            
            # ส่งผลกลับไปให้สรุป
            messages.append({"role": "model", "content": response})
            messages.append({"role": "user", "content": f'<tool_response name="{tool_data["name"]}">{json.dumps(result, ensure_ascii=False)}</tool_response>'})
            
            inputs_v2 = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True, return_dict=True).to(model.device)
            outputs_v2 = model.generate(**inputs_v2, max_new_tokens=512, do_sample=True, temperature=0.7)
            final_ans = tokenizer.decode(outputs_v2[0][inputs_v2['input_ids'].shape[1]:], skip_special_tokens=True)
            print(f"🤖 Agent (ตอบ): {final_ans}")
        else:
            print(f"❌ Failed: โมเดลเรียก Tool ผิด ({tool_data['name']})")
            
    except Exception as e:
        print(f"❌ Error: {e}")

🧪 ทดสอบ Tool ใหม่ที่ไม่เคยเทรน (calculate_installment)
👤 ลูกค้า: ล้อแม็กชุดละ 20,000 บาท ถ้าผ่อน 10 เดือน ตกเดือนละเท่าไหร่ครับ?
🤖 Agent (คิด): <tool_call>{"name": "calculate_installment", "arguments": {"total_price": 20000, "months": 10}}</tool_call>
✅ SUCCESS! โมเดลเรียก Tool ใหม่ได้ถูกต้อง
🤖 Agent (ตอบ): ค่างวดผ่อนเดือนละ 2,100 บาท รวมจ่าย 21,000 บาท คิดดอกเบี้ย 5% ต่อปี
